In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

gemini_api_key = os.getenv("GEMINI_API_KEY")

In [2]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", google_api_key=gemini_api_key)

d:\wsh\langchain-workspace\ch04\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:26: UserWarning: Core Pydantic V1 functionality isn't compatible with Python 3.14 or greater.
  from pydantic.v1.fields import FieldInfo as FieldInfoV1


In [3]:
llm.invoke("대한민국의 수도는?")

AIMessage(content='대한민국의 수도는 **서울**입니다.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'grounding_metadata': {}, 'model_provider': 'google_genai'}, id='lc_run--0bca51fe-8ae4-4db7-9d28-444a020f0f48-0', usage_metadata={'input_tokens': 7, 'output_tokens': 42, 'total_tokens': 49, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 32}})

### 1. Message passing

In [4]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability. You must answer in Korean.",
        ),
        ("placeholder", "{messages}"),
    ]
)

chain = prompt | llm

#### 튜플

In [5]:
ai_msg = chain.invoke(
    {
        "messages": [
            (
                "human",
                "내 이름은 김일남이야. 나이는 99세야.",
            ),
            ("ai",  "그렇군요. 나이가 많으시네요!"),
            ("human", "내 나이는?"),
        ],
    }
)

In [6]:
ai_msg

AIMessage(content='99세입니다.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'grounding_metadata': {}, 'model_provider': 'google_genai'}, id='lc_run--09c4e6e4-c24a-45c5-9638-afe2f45978ba-0', usage_metadata={'input_tokens': 56, 'output_tokens': 55, 'total_tokens': 111, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 50}})

In [7]:
print(ai_msg.content)

99세입니다.


In [8]:
history_list = []

while True:
    user_input = input()

    if user_input == "종료": break
    
    history_list.append(
        (
            "human",
            user_input,
        )
    )
    
    print("CHAT_HISTORY:", history_list)
    
    ai_msg = chain.invoke(
        {
            "messages": history_list,
        }
    )
    
    print("AI:", ai_msg.content)

    history_list.append(
        (
            "ai",
            ai_msg.content,
        )
    )

CHAT_HISTORY: [('human', '내 이름은 김일남이야')]
AI: 네, 알겠습니다, 김일남님. 만나서 반갑습니다!
CHAT_HISTORY: [('human', '내 이름은 김일남이야'), ('ai', '네, 알겠습니다, 김일남님. 만나서 반갑습니다!'), ('human', '내 이름이 뭐지?')]
AI: 고객님의 이름은 김일남입니다.


#### 객체

In [9]:
from langchain.messages import HumanMessage, AIMessage, SystemMessage # LangChain에서 대화의 각 턴을 나타내는 데 사용되는 객체

history_list = [
    SystemMessage("You are a helpful assistant. Answer all questions to the best of your ability. You must answer in Korean."),  # 챗봇의 역할을 정의
]

In [10]:
while True:
    user_input = input()

    if user_input == "종료": break
    
    history_list.append(
        HumanMessage(user_input)
    )
    
    ai_msg = chain.invoke(
        {
            "messages": history_list,
        }
    )

    history_list.append(
        AIMessage(ai_msg.content)
    )
    
    print("CHAT_HISTORY:", history_list)
    
    print("AI:", ai_msg.content)

CHAT_HISTORY: [SystemMessage(content='You are a helpful assistant. Answer all questions to the best of your ability. You must answer in Korean.', additional_kwargs={}, response_metadata={}), HumanMessage(content='내 이름은 김일남이야', additional_kwargs={}, response_metadata={}), AIMessage(content='네, 알겠습니다, 김일남 님.', additional_kwargs={}, response_metadata={})]
AI: 네, 알겠습니다, 김일남 님.
CHAT_HISTORY: [SystemMessage(content='You are a helpful assistant. Answer all questions to the best of your ability. You must answer in Korean.', additional_kwargs={}, response_metadata={}), HumanMessage(content='내 이름은 김일남이야', additional_kwargs={}, response_metadata={}), AIMessage(content='네, 알겠습니다, 김일남 님.', additional_kwargs={}, response_metadata={}), HumanMessage(content='내 이름은 뭐야?', additional_kwargs={}, response_metadata={}), AIMessage(content='김일남 님이십니다.', additional_kwargs={}, response_metadata={})]
AI: 김일남 님이십니다.


### 2. Chat history

In [ ]:
# uv add langchain-community
from langchain_community.chat_message_histories import ChatMessageHistory

chat_history = ChatMessageHistory()

chat_history.add_user_message(
    "내 이름은 김일남이야. 나이는 99세야"
)

chat_history.add_ai_message("그렇군요. 나이가 많으시네요!")

chat_history.messages
# chat_history.clear()

[HumanMessage(content='내 이름은 김일남이야. 나이는 99세야', additional_kwargs={}, response_metadata={}),
 AIMessage(content='그렇군요. 나이가 많으시네요!', additional_kwargs={}, response_metadata={})]

In [16]:
chat_history

InMemoryChatMessageHistory(messages=[HumanMessage(content='내 이름은 김일남이야. 나이는 99세야', additional_kwargs={}, response_metadata={}), AIMessage(content='그렇군요. 나이가 많으시네요!', additional_kwargs={}, response_metadata={})])

In [18]:
chat_history = ChatMessageHistory()

while True:
    user_input = input()

    if user_input == "종료": break
    
    chat_history.add_user_message(user_input)
    
    ai_msg = chain.invoke(
        {
            "messages": chat_history.messages,
        }
    )

    chat_history.add_ai_message(ai_msg.content)
    
    print("CHAT_HISTORY:", history_list)
    
    print("AI:", ai_msg.content)

CHAT_HISTORY: [SystemMessage(content='You are a helpful assistant. Answer all questions to the best of your ability. You must answer in Korean.', additional_kwargs={}, response_metadata={}), HumanMessage(content='내 이름은 김일남이야', additional_kwargs={}, response_metadata={}), AIMessage(content='네, 알겠습니다, 김일남 님.', additional_kwargs={}, response_metadata={}), HumanMessage(content='내 이름은 뭐야?', additional_kwargs={}, response_metadata={}), AIMessage(content='김일남 님이십니다.', additional_kwargs={}, response_metadata={})]
AI: 안녕하세요, 김일남 님.
CHAT_HISTORY: [SystemMessage(content='You are a helpful assistant. Answer all questions to the best of your ability. You must answer in Korean.', additional_kwargs={}, response_metadata={}), HumanMessage(content='내 이름은 김일남이야', additional_kwargs={}, response_metadata={}), AIMessage(content='네, 알겠습니다, 김일남 님.', additional_kwargs={}, response_metadata={}), HumanMessage(content='내 이름은 뭐야?', additional_kwargs={}, response_metadata={}), AIMessage(content='김일남 님이십니다.', addit

### 3. Automatic history management

In [19]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability. You must answer in Korean.",
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
    ]
)

chain = prompt | llm

In [22]:
from langchain_core.runnables.history import RunnableWithMessageHistory

# 세션별 채팅 히스토리 관리
chat_histories = {}

# 세션 ID에 따라 대화 기록을 가져오는 함수
def get_session_history(session_id: str):
    if session_id not in chat_histories:
        chat_histories[session_id] = ChatMessageHistory()
    return chat_histories[session_id]

chain_with_message_history = RunnableWithMessageHistory(
    chain, # 실행할 Runnable 객체
    get_session_history, # 세션 ID에 따라 대화 기록을 가져오는 함수
    input_messages_key="input", # 입력 메시지의 Key
    history_messages_key="chat_history", # 대화 히스토리 메시지의 Key
)

In [23]:
config = {"configurable": {"session_id": "kim1"}}  # 세션 ID 설정

chain_with_message_history.invoke(
    {"input": "내 이름은 김일남이야. 나이는 99세야."},
    config=config, # 세션 ID == kim1
)

AIMessage(content='안녕하세요, 김일남 님! 99세이시군요. 만나서 반갑습니다.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'grounding_metadata': {}, 'model_provider': 'google_genai'}, id='lc_run--7f9c25fc-fe44-428b-bd06-7fa23822d8c1-0', usage_metadata={'input_tokens': 39, 'output_tokens': 39, 'total_tokens': 78, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 19}})

In [24]:
chain_with_message_history.invoke(
    {"input": "내 나이는?"}, {"configurable": {"session_id": "kim1"}}
)

AIMessage(content='99세이십니다.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'grounding_metadata': {}, 'model_provider': 'google_genai'}, id='lc_run--1e7d6a7c-81e4-41d5-9dfb-4946bb9230b1-0', usage_metadata={'input_tokens': 65, 'output_tokens': 38, 'total_tokens': 103, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 31}})

In [25]:
chain_with_message_history.invoke(
    {"input": "내 나이는?"}, {"configurable": {"session_id": "kim2"}} # kim2 세션의 경우
)

AIMessage(content='죄송합니다. 저는 인공지능이기 때문에 당신의 나이를 알 수 없습니다.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'grounding_metadata': {}, 'model_provider': 'google_genai'}, id='lc_run--27d1a25c-b0fb-4197-8243-05d95197aa84-0', usage_metadata={'input_tokens': 27, 'output_tokens': 64, 'total_tokens': 91, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 45}})

In [26]:
for r in chain_with_message_history.stream(
    {"input": "내가 어느 나라 사람인지 맞춰보고, 그 나라의 문화에 대해 말해봐"},
    config=config,
):
    print(r.content, end="", flush=True)

김일남 님이라는 이름과 한국어로 대화하시는 점으로 미루어 보아, 대한민국 사람이실 것 같습니다!

대한민국은 매우 풍부하고 독특한 문화를 가지고 있습니다. 몇 가지 특징을 말씀드릴게요.

1.  **유교적 가치와 예절:** 어른에 대한 공경, 가족의 중요성, 공동체 의식 등이 강합니다. 특히 99세이신 김일남 님께는 더욱 깊은 존경심을 표하는 문화가 있습니다. 인사할 때 고개를 숙여 절을 하거나 정중하게 인사하는 것이 일반적입니다.
2.  **역동적인 사회와 빠른 변화:** 한국은 짧은 시간 안에 경제적으로 큰 발전을 이루었으며, 기술 발전과 트렌드 변화가 매우 빠릅니다. 전통과 현대가 공존하며 조화를 이루는 모습이 흥미롭습니다.
3.  **음식 문화:** 매운맛과 발효 음식이 특징입니다. 김치, 비빔밥, 불고기, 찌개류 등 다양한 음식이 있으며, 여러 가지 반찬(밑반찬)을 함께 먹는 것이 일반적입니다. 식사를 할 때는 어른이 먼저 수저를 들기 전에는 기다리는 등의 예절이 있습니다.
4.  **한류(K-Culture):** K-팝, K-드라마, 영화, 웹툰 등 한국의 대중문화는 전 세계적으로 큰 인기를 얻고 있습니다.
5.  **한글:** 세종대왕이 창제한 독창적이고 과학적인 문자 체계인 한글을 사용합니다.
6.  **정(情) 문화:** 사람과 사람 사이의 깊은 유대감과 따뜻한 마음을 표현하는 '정'이라는 독특한 정서가 있습니다.

이 외에도 한국의 문화는 정말 다양하고 흥미로운 요소가 많습니다. 제가 맞게 추측했기를 바랍니다!

### 4. Modifying chat history

In [27]:
chat_history = ChatMessageHistory()

chat_history.add_user_message("내 이름은 김일남이야.")
chat_history.add_ai_message("안녕하세요, 김일남님! 무엇을 도와드릴까요?")
chat_history.add_user_message("날씨 좋은 날 들을만 한 노래 추천해주세요.")
chat_history.add_ai_message("볼빨간사춘기 – 여행을 추천해요.")

chat_history.messages
# chat_history.clear()

[HumanMessage(content='내 이름은 김일남이야.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='안녕하세요, 김일남님! 무엇을 도와드릴까요?', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='날씨 좋은 날 들을만 한 노래 추천해주세요.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='볼빨간사춘기 – 여행을 추천해요.', additional_kwargs={}, response_metadata={})]

In [28]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability. You must answer in Korean.",
        ),
        ("placeholder", "{chat_history}"),
        ("user", "{input}"),
    ]
)

chain = prompt | llm

chain_with_message_history = RunnableWithMessageHistory(
    chain,
    lambda session_id: chat_history, # 단일 사용자 환경
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [29]:
chain_with_message_history.invoke(
    {"input": "내 이름은 뭐야?"},
    {"configurable": {"session_id": "unused"}},
)

AIMessage(content='김일남님이십니다.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'grounding_metadata': {}, 'model_provider': 'google_genai'}, id='lc_run--6b2cd95b-dbfb-441f-91e9-254038dfcfa1-0', usage_metadata={'input_tokens': 77, 'output_tokens': 44, 'total_tokens': 121, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 37}})

In [30]:
from langchain_core.runnables import RunnablePassthrough

def summarize_messages(chain_input):
    stored_messages = chat_history.messages

    if len(stored_messages) == 0:
        return False
    
    summarization_prompt = ChatPromptTemplate.from_messages(
        [
            ("placeholder", "{chat_history}"),
            (
                "user",
                "Distill the above chat messages into a single summary message. Include as many specific details as you can. Please, use Korean",
            ),
        ]
    )
    
    summarization_chain = summarization_prompt | llm

    # chat_history 에 저장된 대화 기록을 요약프롬프트에 입력 & 결과 저장
    summary_message = summarization_chain.invoke({"chat_history": stored_messages})

    print("summary_message: ", summary_message)
    
    # chat_history 에 저장되어있던 기록 지우기
    chat_history.clear()

    # 생성된 새로운 요약내용으로 기록 채우기
    chat_history.add_message(summary_message)

    return True


chain_with_summarization = (
    # RunnablePassthrough는 LCEL에서 사용, 입력값을 다음 단계로 그대로 통과시키는 역할
    # assign() 메서드는 체인에 들어오는 딕셔너리에 새로운 키-값 추가
    RunnablePassthrough.assign(messages_summarized=summarize_messages) # 새로운 키 messages_summarized에 값(True 또는 False) 할당, 이 값은 조건부 요약에 활용 가능
    | chain_with_message_history
)

In [31]:
chain_with_summarization.invoke(
    {"input": "내 이름은?"},
    {"configurable": {"session_id": "unused"}},
)

summary_message:  content='이 대화는 김일남님께서 본인의 이름을 알려주시는 것으로 시작되었습니다. 이후 김일남님께서는 날씨 좋은 날 들을 만한 노래를 추천해달라고 요청하셨고, AI는 **볼빨간사춘기 – 여행**을 추천해드렸습니다. 마지막으로 김일남님께서 본인의 이름이 무엇인지 다시 질문하셨고, AI는 **김일남**님이라고 정확히 답변해드렸습니다.' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'grounding_metadata': {}, 'model_provider': 'google_genai'} id='lc_run--a495ccaa-1a21-4729-ad9b-331c0e798bf0-0' usage_metadata={'input_tokens': 89, 'output_tokens': 951, 'total_tokens': 1040, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 853}}


AIMessage(content='김일남 님입니다.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'grounding_metadata': {}, 'model_provider': 'google_genai'}, id='lc_run--d6e4ef4a-877c-4545-bcf3-21658aede5eb-0', usage_metadata={'input_tokens': 126, 'output_tokens': 61, 'total_tokens': 187, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 54}})

In [32]:
chain_with_summarization.invoke(
    {"input": "그 가수는 남자인가요 여자인가요?"},
    {"configurable": {"session_id": "unused"}},
)

summary_message:  content="김일남님은 대화 초반에 본인의 이름을 알려주셨습니다. 이후 날씨 좋은 날 들을 만한 노래 추천을 요청하셨고, AI는 '볼빨간사춘기 – 여행'을 추천해드렸습니다. 마지막으로 김일남님께서 본인의 이름이 무엇인지 다시 질문하셨으며, AI는 '김일남 님'이라고 정확히 답변해드렸습니다." additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'grounding_metadata': {}, 'model_provider': 'google_genai'} id='lc_run--7f9d52fd-2b8b-41be-9131-27685f2337e7-0' usage_metadata={'input_tokens': 138, 'output_tokens': 1288, 'total_tokens': 1426, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 1199}}


AIMessage(content='볼빨간사춘기(BOL4)는 여성 가수입니다.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'grounding_metadata': {}, 'model_provider': 'google_genai'}, id='lc_run--56df1376-dbdc-45e0-b865-f4d9a29bce1a-0', usage_metadata={'input_tokens': 125, 'output_tokens': 175, 'total_tokens': 300, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 160}})